In [3]:
%load_ext autoreload
%autoreload 2
import util
import nest_asyncio
nest_asyncio.apply()
import multiprocessing
# if context not already set 
if multiprocessing.get_start_method(allow_none=True) is None:
    multiprocessing.set_start_method("fork")
import pandas as pd
import stan
import numpy as np
from scipy.stats import pearsonr

from scipy.special import expit
import matplotlib.pyplot as plt
import arviz as az
import random

import json 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Emma notes (8/22):

In general, let's build complexity iteratively. Start by getting reasonable results without worrying about ICAR prior/smoothing. Then use standard ICAR prior (with weight 0.5). Then use full CAR (maybe). Data generation code is reviewed + model without proper CAR prior is reviewed. Another thing it might be nice to implement at some point is using the information about where the annotated images are (i.e., what Census tracts). Could incorporate this as a multinomial (potentially?) 

Model with simple L2 smoothing (or no smoothing at all - stan_code_with_weighted_ICAR_prior):

1. Consistently recovers parameters for realistic parameter settings (with no smoothing)
2. Reviewed Stan code and looks good. 
3. ALso implemented L2 regularization for adjacent Census tracts. This isn't actually the "proper" way to do it, but might be useful on real data. Haven't tested how this performs. 

Model wih full CAR prior (stan_code_proper_car_prior):

1. Haven't reviewed this or verified it recovers correct params (recently; I think I did a while ago). When you do review, don't need to re-review all the Bayesian conditioning math; it should be pretty similar to the old code. 


In [4]:
icar_prior_setting = 'none'
annotations_have_locations = True
use_simulated_data = False
assert icar_prior_setting in ['none', 'cheating', 'proper', 'just_model_p_y']
stan_code_with_weighted_ICAR_prior = open('stan_models/weighted_ICAR_prior.stan').read()
stan_code_proper_car_prior = open('stan_models/proper_car_prior.stan').read()
stan_model_uniform_p_y = open('stan_models/uniform_p_y_prior_just_for_debugging.stan').read()
stan_code_with_weighted_ICAR_prior_annotations_have_locations = open('stan_models/weighted_ICAR_prior_annotations_have_locations.stan').read()

OBSERVED_DATA_FLAG = 'simulated' if use_simulated_data else 'real'

for i in range(3):
    NUM_WARMUP = 1000#3000
    NUM_SAMPLES = 1500
    if use_simulated_data:
        print("Using simulated data")
        N = 1000
        data_to_use = util.generate_simulated_data(N=N, 
                                                images_per_location=1000, 
                                                total_annotated_classified_negative=500, 
                                                total_annotated_classified_positive=500, 
                                                icar_prior_setting=icar_prior_setting, 
                                                annotations_have_locations=annotations_have_locations)
    else:
        data_to_use = util.read_real_data(single_compartment_for_debugging=False, annotations_have_locations=annotations_have_locations)
    if icar_prior_setting == 'proper':
        raise Exception("Haven't verified that this model actually works! Need to review it / check on simulated data. No need to review the parts which are identical to the other model.")
        W = np.zeros((N, N))
        
        for i in range(len(simulated_data['observed_data']['node1'])):
            W[simulated_data['observed_data']['node1'][i] - 1, 
                                simulated_data['observed_data']['node2'][i] - 1] = 1
            W[simulated_data['observed_data']['node2'][i] - 1, 
                                simulated_data['observed_data']['node1'][i] - 1] = 1
        del simulated_data['observed_data']['node1']
        del simulated_data['observed_data']['node2']
        simulated_data['observed_data']['W'] = W
        simulated_data['observed_data']['W_n'] = int(W.sum() / 2)
        model = stan.build(stan_code_proper_car_prior, data=simulated_data['observed_data'])
    elif icar_prior_setting == 'cheating':
        data_to_use['observed_data']['use_ICAR_prior'] = 1
        data_to_use['observed_data']['ICAR_prior_weight'] = 0.5
        if annotations_have_locations:
            model = stan.build(stan_code_with_weighted_ICAR_prior_annotations_have_locations, data=data_to_use['observed_data'])
        else:
            model = stan.build(stan_code_with_weighted_ICAR_prior, data=data_to_use['observed_data'])
    elif icar_prior_setting == 'none':
        data_to_use['observed_data']['use_ICAR_prior'] = 0
        data_to_use['observed_data']['ICAR_prior_weight'] = 0
        print(data_to_use['observed_data'].keys())
        if annotations_have_locations:
            # write jsonified observed data to file for debugging
            # need to convert numpy arrays to lists
            observed_data_copy = data_to_use['observed_data'].copy()
            # observed_data_copy is a dict 
            for k in observed_data_copy.keys():
                if isinstance(observed_data_copy[k], np.ndarray):
                    observed_data_copy[k] = observed_data_copy[k].tolist()
                # serialized int64 
                if isinstance(observed_data_copy[k], np.int64):
                    observed_data_copy[k] = int(observed_data_copy[k])
                # serialize nd arrays with int 64 elements 
                if isinstance(observed_data_copy[k], list):
                    for i in range(len(observed_data_copy[k])):
                        if isinstance(observed_data_copy[k][i], np.int64):
                            observed_data_copy[k][i] = int(observed_data_copy[k][i])

            with open(f'observed_data_{OBSERVED_DATA_FLAG}_raw.json', 'w') as f:
                json.dump(observed_data_copy, f)

            # also, dump the sums of each array 
            with open(f'observed_data_{OBSERVED_DATA_FLAG}_sums.json', 'w') as f:
                sums = {}
                for k in observed_data_copy.keys():
                    sums[k] = float(np.sum(observed_data_copy[k]))
                json.dump(sums, f)

           
            
            model = stan.build(stan_code_with_weighted_ICAR_prior_annotations_have_locations, data=data_to_use['observed_data'])
        else:
            model = stan.build(stan_code_with_weighted_ICAR_prior, data=data_to_use['observed_data'])
    elif icar_prior_setting == 'just_model_p_y':
        del data_to_use['observed_data']['node1']
        del data_to_use['observed_data']['node2']
        del data_to_use['observed_data']['N_edges']
        model = stan.build(stan_model_uniform_p_y, data=data_to_use['observed_data'])
    else:
        raise ValueError("Invalid icar_prior_options", icar_prior_setting)
    fit = model.sample(num_chains=4, num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES)
    df = fit.to_frame()
    if icar_prior_setting == 'just_model_p_y':
        print(az.summary(fit, var_names=['p_y_1_given_y_hat_1', 'p_y_1_given_y_hat_0', 
                                        'p_y_hat_1_given_y_1', 'p_y_hat_1_given_y_0', 
                                        'empirical_p_yhat', 'p_y']))
    else:
        print(az.summary(fit, var_names=['p_y_hat_1_given_y_1', 'p_y_hat_1_given_y_0', 'phi_offset', 
                                    'p_y_1_given_y_hat_1', 'p_y_1_given_y_hat_0', 
                                    'empirical_p_yhat', 'p_y']))
        
    if use_simulated_data:
        inferred_p_y = [df[f'p_y.{i}'].mean() for i in range(1, N + 1)]
        plt.scatter(data_to_use['parameters']['p_Y'], inferred_p_y)
        plt.title("True vs. inferred p_Y, r = %.2f" %
                pearsonr(data_to_use['parameters']['p_Y'], inferred_p_y)[0])
        max_val = max(max(data_to_use['parameters']['p_Y']), max(inferred_p_y))
        plt.xlabel("True p_Y")
        plt.ylabel("Inferred p_Y")
        plt.plot([0, max_val], [0, max_val], 'r--')
        plt.xlim([0, max_val])
        plt.ylim([0, max_val])
        plt.figure(figsize=[12, 3])


        if icar_prior_setting == 'proper':
            param_names = ['p_y_hat_1_given_y_1', 'p_y_hat_1_given_y_0', 
            'p_y_1_given_y_hat_1', 'p_y_1_given_y_hat_0', 
            'phi_offset', 'alpha', 'tau']
        elif icar_prior_setting == 'just_model_p_y':
            param_names = ['p_y_hat_1_given_y_1', 'p_y_hat_1_given_y_0', 
            'p_y_1_given_y_hat_1', 'p_y_1_given_y_hat_0', 
            'phi_offset']
        else:
            param_names = ['p_y_hat_1_given_y_1', 'p_y_hat_1_given_y_0', 
            'p_y_1_given_y_hat_1', 'p_y_1_given_y_hat_0']
        for k in param_names:
            plt.subplot(1, len(param_names), param_names.index(k) + 1)
            # histogram of posterior samples
            plt.hist(df[k], bins=50, density=True)
            plt.title(k)
            plt.axvline(data_to_use['parameters'][k], color='red')
        plt.show()
    else:
        empirical_p_yhat = data_to_use['observed_data']['n_classified_positive_by_area'] / data_to_use['observed_data']['n_images_by_area']
        print("Warning: %i of %i empirical p_yhat values are 0; these are being ignored" % (sum(np.isnan(empirical_p_yhat)), len(empirical_p_yhat)))
        inferred_p_y = np.array([df[f'p_y.{i}'].mean() for i in range(1, len(empirical_p_yhat) + 1)])
        inferred_p_y_CIs = [df[f'p_y.{i}'].quantile([0.025, 0.975]) for i in range(1, len(empirical_p_yhat) + 1)]
        n_images_by_area = data_to_use['observed_data']['n_images_by_area']
        # make errorbar plot
        image_cutoff = 200

        plt.errorbar(empirical_p_yhat[n_images_by_area >= image_cutoff], inferred_p_y[n_images_by_area >= image_cutoff], 
                    yerr=np.array(inferred_p_y_CIs)[n_images_by_area >= image_cutoff].T, fmt='o', 
                    color='blue', 
                    ecolor='lightgray', elinewidth=1, capsize=3, alpha=0.5, 
                    label="n_images_by_area >= %i" % image_cutoff)

        plt.errorbar(empirical_p_yhat[n_images_by_area < image_cutoff], inferred_p_y[n_images_by_area < image_cutoff], 
                    yerr=np.array(inferred_p_y_CIs)[n_images_by_area < image_cutoff].T, fmt='o', 
                    color='red', 
                    ecolor='lightgray', elinewidth=1, capsize=3, alpha=0.5, 
                    label="n_images_by_area < %i" % image_cutoff)
        plt.legend()

        # plot prior on p_y as vertical line. 
        phi_offset = df['phi_offset'].mean()
        plt.axhline(expit(phi_offset), color='black', linestyle='--')
        is_nan = np.isnan(empirical_p_yhat)
        plt.title("Correlation between empirical $p(\\hat y = 1)$ and inferred $p(y = 1)$, r = %.2f" % pearsonr(empirical_p_yhat[~is_nan], inferred_p_y[~is_nan])[0])
        plt.xlabel("empirical $p(\\hat y = 1)$")
        plt.ylabel("inferred $p(y = 1)$")
        # logarithmic axes
        plt.xscale('log')
        plt.yscale('log')

        plt.savefig(f"correlation_{icar_prior_setting}_{annotations_have_locations}_{i}.png")
        plt.close()




dict_keys(['N', 'N_edges', 'node1', 'node2', 'n_images_by_area', 'n_classified_positive_by_area', 'n_classified_positive_annotated_positive_by_area', 'n_classified_positive_annotated_negative_by_area', 'n_classified_negative_annotated_negative_by_area', 'n_classified_negative_annotated_positive_by_area', 'n_non_annotated_by_area', 'n_non_annotated_by_area_classified_positive', 'use_ICAR_prior', 'ICAR_prior_weight'])
Building...



Building: found in cache, done.Messages from stanc:
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
Sampling:   0%
Sampling:   0% (1/10000)
Sampling:   0% (2/10000)
Sampling:   0% (3/10000)
Sampling:   0% (4/10000)
Sampling:   1% (103/10000)
Sampling:   2% (203/10000)
Sampling:   3% (303/10000)
Sampling:   4% (403/10000)
Sampling:   5% (502/10000)
Sampling:   6% (602/10000)
Sampling:   7% (701/10000)
Sampling:   8% (800/10000)
Sampling:   9% (900/10000)
Sampling:  10% (1000/10000)
Sampling:  11% (1100/10000)
Sampling:  12% (1200/10000)
Sampling:  13% (1300/10000)
Sampling:  14% (1400/10000)
Sampling:  15% (1500/10000)
Sampling:  16% (1600/10000)
Sampling:  17% (1700/10000)
Sampling:  18% (1800/10000)
Sampling:  19% (1900/10000)
Sampling:  20% (2000/10000)
Sampling:  21% (2100/10000)
Sampling:  22% (2200/10000)
Sampling:  23% (2301/10000)
Sampling:  24% (2401/10000)
Sampling:  25% (2501/10000)
Sampling:  26

                      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
p_y_hat_1_given_y_1  0.018  0.002   0.016    0.021      0.000    0.000   
p_y_hat_1_given_y_0  0.000  0.000   0.000    0.000      0.000    0.000   
phi_offset          -3.076  0.100  -3.258   -2.887      0.005    0.004   
p_y_1_given_y_hat_1  0.996  0.003   0.991    0.999      0.000    0.000   
p_y_1_given_y_hat_0  0.085  0.007   0.072    0.098      0.000    0.000   
...                    ...    ...     ...      ...        ...      ...   
p_y[2322]            0.060  0.058   0.001    0.166      0.001    0.001   
p_y[2323]            0.100  0.080   0.005    0.248      0.001    0.001   
p_y[2324]            0.030  0.024   0.002    0.073      0.000    0.000   
p_y[2325]            0.051  0.049   0.002    0.136      0.001    0.001   
p_y[2326]            0.107  0.085   0.005    0.268      0.001    0.001   

                     ess_bulk  ess_tail  r_hat  
p_y_hat_1_given_y_1     431.0    1141.0   1.00  
p_y_hat_1_giv

/tmp/ipykernel_3217425/1006087040.py:132: RuntimeWarning: invalid value encountered in divide
  empirical_p_yhat = data_to_use['observed_data']['n_classified_positive_by_area'] / data_to_use['observed_data']['n_images_by_area']


dict_keys(['N', 'N_edges', 'node1', 'node2', 'n_images_by_area', 'n_classified_positive_by_area', 'n_classified_positive_annotated_positive_by_area', 'n_classified_positive_annotated_negative_by_area', 'n_classified_negative_annotated_negative_by_area', 'n_classified_negative_annotated_positive_by_area', 'n_non_annotated_by_area', 'n_non_annotated_by_area_classified_positive', 'use_ICAR_prior', 'ICAR_prior_weight'])
Building...


...

/tmp/ipykernel_3217425/1006087040.py:132: RuntimeWarning: invalid value encountered in divide
  empirical_p_yhat = data_to_use['observed_data']['n_classified_positive_by_area'] / data_to_use['observed_data']['n_images_by_area']


dict_keys(['N', 'N_edges', 'node1', 'node2', 'n_images_by_area', 'n_classified_positive_by_area', 'n_classified_positive_annotated_positive_by_area', 'n_classified_positive_annotated_negative_by_area', 'n_classified_negative_annotated_negative_by_area', 'n_classified_negative_annotated_positive_by_area', 'n_non_annotated_by_area', 'n_non_annotated_by_area_classified_positive', 'use_ICAR_prior', 'ICAR_prior_weight'])
Building...



Building: found in cache, done.Messages from stanc:
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
Sampling:   0%
Sampling:   0% (1/10000)
Sampling:   0% (2/10000)
Sampling:   0% (3/10000)
Sampling:   0% (4/10000)
Sampling:   1% (103/10000)
Sampling:   2% (203/10000)
Sampling:   3% (303/10000)
Sampling:   4% (403/10000)
Sampling:   5% (502/10000)
Sampling:   6% (602/10000)
Sampling:   7% (701/10000)
Sampling:   8% (800/10000)
Sampling:   9% (900/10000)
Sampling:  10% (1000/10000)
Sampling:  11% (1100/10000)
Sampling:  12% (1200/10000)
Sampling:  13% (1300/10000)
Sampling:  14% (1400/10000)
Sampling:  15% (1500/10000)
Sampling:  16% (1600/10000)
Sampling:  17% (1700/10000)
Sampling:  18% (1800/10000)
Sampling:  19% (1900/10000)
Sampling:  20% (2000/10000)
Sampling:  21% (2100/10000)
Sampling:  22% (2200/10000)
Sampling:  23% (2300/10000)
Sampling:  24% (2400/10000)
Sampling:  25% (2501/10000)
Sampling:  26

                      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
p_y_hat_1_given_y_1  0.018  0.002   0.016    0.021      0.000    0.000   
p_y_hat_1_given_y_0  0.000  0.000   0.000    0.000      0.000    0.000   
phi_offset          -3.076  0.100  -3.258   -2.887      0.005    0.004   
p_y_1_given_y_hat_1  0.996  0.003   0.991    0.999      0.000    0.000   
p_y_1_given_y_hat_0  0.085  0.007   0.072    0.098      0.000    0.000   
...                    ...    ...     ...      ...        ...      ...   
p_y[2322]            0.060  0.058   0.001    0.166      0.001    0.001   
p_y[2323]            0.100  0.080   0.005    0.248      0.001    0.001   
p_y[2324]            0.030  0.024   0.002    0.073      0.000    0.000   
p_y[2325]            0.051  0.049   0.002    0.136      0.001    0.001   
p_y[2326]            0.107  0.085   0.005    0.268      0.001    0.001   

                     ess_bulk  ess_tail  r_hat  
p_y_hat_1_given_y_1     431.0    1141.0   1.00  
p_y_hat_1_giv

/tmp/ipykernel_3217425/1006087040.py:132: RuntimeWarning: invalid value encountered in divide
  empirical_p_yhat = data_to_use['observed_data']['n_classified_positive_by_area'] / data_to_use['observed_data']['n_images_by_area']
